In [1014]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [967]:
train=pd.read_csv('train.csv',nrows=2_00_000)
test=pd.read_csv('test.csv',parse_dates=True)


In [968]:
def distance(df):
    df['abs_lat']=(df.dropoff_latitude-df.pickup_latitude).abs()
    df['abs_lon']=(df.dropoff_longitude-df.pickup_longitude).abs()

In [969]:
distance(train)
distance(test)

In [970]:
train=train.dropna(how='any',axis=0)
#train.info()

In [971]:
#train.corr()
#test.info()

In [972]:
train=train[train.fare_amount>0]
#train.info()
train=train[(train.abs_lat<1)&(train.abs_lon<1)]
#train.describe()

In [973]:
#train.loc[(train.fare_amount>200)]

In [974]:
#train.loc[(train['pickup_longitude']==0)&(train['pickup_latitude']==0)]

In [975]:
#train.loc[(train['pickup_longitude']==0)]
#train[(train['pickup_longitude']==0)&(train['pickup_latitude']==0)].count()
#test[(test['pickup_longitude']==0)&(test['pickup_latitude']==0)].count()

In [976]:
#train.loc[(train['pickup_latitude']==0)]
#test.loc[test.passenger_count==0]

In [977]:
train=train[(train['pickup_longitude']!=0)]
train=train[train.passenger_count!=0]

In [978]:
#train.describe()
#train.loc[(train.pickup_longitude>0)&(train.pickup_latitude<0)]

In [979]:
cond=(train.pickup_longitude>0) & (train.pickup_latitude<0)
train.loc[cond,['pickup_longitude','pickup_latitude']]=train.loc[cond,['pickup_latitude','pickup_longitude']].values
train.loc[cond,['dropoff_longitude','dropoff_latitude']]=train.loc[cond,['dropoff_latitude','dropoff_longitude']].values

In [980]:
#test.loc[(test.pickup_longitude>0)]
#train.loc[(train.pickup_latitude<0)]
#train.loc[(train.pickup_longitude>0)]
#train.loc[(train.dropoff_longitude>0)]
train=train[(train.pickup_longitude<-10)]

In [981]:
#plt.scatter(train.pickup_longitude,train.pickup_latitude)

In [982]:
#plt.scatter(train.dropoff_longitude,train.dropoff_latitude)
#plt.scatter(test.pickup_longitude,test.pickup_latitude)

In [983]:
#train.loc[(train.dropoff_longitude>0)&(train.dropoff_latitude<0)]

In [984]:
train=train[(train.fare_amount>0.1)]
#train.groupby('passenger_count')['fare_amount'].min()
#train['passenger_count'].value_counts()
train['pickup_datetime']=pd.to_datetime(train['pickup_datetime'])

In [1007]:
#plt.hist(train['pickup_datetime'].dt.hour,24,rwidth=0.75)
train.groupby(train['pickup_datetime'].dt.hour)['fare_amount'].mean()


pickup_datetime
0     11.584566
1     11.584514
2     11.595550
3     11.991769
4     13.547048
5     14.881881
6     12.265310
7     11.003020
8     10.837789
9     10.795681
10    10.861286
11    11.157240
12    11.079903
13    11.570448
14    11.912628
15    12.096309
16    11.923413
17    11.266933
18    10.913207
19    10.607643
20    10.870132
21    11.082884
22    11.249585
23    11.526426
Name: fare_amount, dtype: float64

In [1006]:
train['pickup_hour']=train['pickup_datetime'].dt.hour
train['pickup_year']=train['pickup_datetime'].dt.year
x=train.copy()
x=x.drop(['key','pickup_datetime','fare_amount'],1)
y=train['fare_amount'].copy()

test['pickup_datetime']=pd.to_datetime(test['pickup_datetime'])
test['pickup_hour']=test['pickup_datetime'].dt.hour
test['pickup_year']=test['pickup_datetime'].dt.year
x_test=test.drop(['key','pickup_datetime'],axis=1)
key=test['key']
x.head()

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
lenc=LabelEncoder()
dummy=OneHotEncoder(categorical_features=[-1])
x['pickup_year']=lenc.fit_transform(x['pickup_year'])
x_test['pickup_year']=lenc.transform(x_test['pickup_year'])

x=dummy.fit_transform(x).toarray()
x_test=dummy.transform(x_test).toarray()
x=x[:,:-1]
x_test=x_test[:,:-1]

In [1000]:
sc_x = StandardScaler()
col=['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count',
     'abs_lat','abs_lon','pickup_hour','pickup_year']
cols=['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count',
     'abs_lat','abs_lon','pickup_hour','pickup_year']
x=sc_x.fit_transform(x)
x_test=sc_x.transform(x_test)
#x_test.loc[(x_test.pickup_year==0)]

In [988]:
"""x=train.drop(['fare_amount'],axis=1)
y=train['fare_amount'].copy()
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)"""

"x=train.drop(['fare_amount'],axis=1)\ny=train['fare_amount'].copy()\nfrom sklearn.model_selection import train_test_split\nx_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)"

In [1001]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()

In [990]:
reg.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [991]:
from sklearn.model_selection import cross_val_score,KFold,cross_val_predict
kf=KFold(n_splits=10,random_state=42)
score=cross_val_score(reg,x,y,cv=kf,scoring='neg_mean_squared_error')
print(score)

[-37.82598705 -30.38994913 -39.54461335 -29.6519092  -30.56376191
 -45.12929481 -33.77481416 -30.46921541 -31.43881653 -32.1775087 ]


In [992]:
y_pred=reg.predict(x_test)
y_pred

array([10.15555177, 11.44754396,  5.61558137, ..., 49.97642121,
       21.24483094,  9.01918397])

In [993]:
texi0=pd.DataFrame({'key':key,'fare_amount':y_pred})
texi0.to_csv('texi0.csv',index=False)

In [1002]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(degree=2)
x_poly=poly.fit_transform(x)
x_test_poly=poly.transform(x_test)
x_poly
poly_reg=LinearRegression()
poly_reg.fit(x_poly,y)
print(cross_val_score(poly_reg,x_poly,y,cv=10
                      ,scoring='neg_mean_squared_error'))

[-21.81096762 -24.23765395 -36.58073628 -24.96430992 -23.51260852
 -34.19491167 -21.05069065 -22.9547377  -22.08034101 -27.67116002]


In [1003]:
y_pred1=poly_reg.predict(x_test_poly)
y_pred1

array([ 9.4432373 , 10.2199707 ,  5.10913086, ..., 56.47265625,
       23.96014404,  8.12811279])

In [1004]:
texi1=pd.DataFrame({'key':key,'fare_amount':y_pred1})
texi1.to_csv('texi1.csv',index=False)

In [ ]:
a= np.random.rand(2,3)
b=np.random.rand(2,1)
c=a*b
print(c)

In [1027]:
a = np.random.rand(12288, 150) # a.shape = (12288, 150)
b = np.random.rand(150, 45) # b.shape = (150, 45)
c = np.dot(a,b)
print (c)

[[35.23829338 40.48499384 43.20010923 ... 33.70209296 37.38111559
  38.56128116]
 [35.03784902 39.02783458 40.88207852 ... 32.33004079 34.24666251
  32.96597712]
 [41.31128526 41.81257439 46.12919184 ... 37.91166594 40.18273027
  37.30918722]
 ...
 [32.7303108  35.30990205 38.95436895 ... 31.61500537 32.87119361
  33.58734933]
 [38.68771007 37.67797761 43.25328797 ... 32.51256286 37.39581806
  35.49085514]
 [38.28402186 38.3234941  42.95929016 ... 33.46260437 37.60287177
  36.25947465]]
